In [25]:
import sys
sys.path.append(r"D:\MINJI\NETWORK RELIABILITY\BNS-JT-python")

import networkx as nx
import matplotlib.pyplot as plt
from BNS_JT import cpm, variable, operation
from networkx.algorithms.flow import shortest_augmenting_path
from BNS_JT import brc
import numpy as np

In [26]:
# Node and edge data
nodes = {
    "n1": (0, 0),
    "n2": (10, 10),
    "n3": (10, -10),
    "n4": (20, 0)
}

edges = {  
    "e1": ("n1", "n2"),   
    "e2": ("n2", "n1"),   
    "e3": ("n1", "n3"),   
    "e4": ("n3", "n1"),   
    "e5": ("n2", "n3"),   
    "e6": ("n3", "n2"),   
    "e7": ("n2", "n4"),   
    "e8": ("n4", "n2"),   
    "e9": ("n3", "n4"),   
    "e10": ("n4", "n3")   
}

arcs = [(u, v) for _, (u, v) in edges.items()]

# Probability of failure/survival (failure: 0, survival: 1)
probs = {
    'e1': {0: 0.01, 1: 0.99}, 
    'e2': {0: 0.01, 1: 0.99}, 
    'e3': {0: 0.01, 1: 0.99},
    'e4': {0: 0.01, 1: 0.99},
    'e5': {0: 0.05, 1: 0.95},
    'e6': {0: 0.05, 1: 0.95}, 
    'e7': {0: 0.05, 1: 0.95}, 
    'e8': {0: 0.05, 1: 0.95},
    'e9': {0: 0.10, 1: 0.90}, 
    'e10': {0: 0.10, 1: 0.90}
}

# Initial intact capacity
intact_capacity = { 
    "e1": 15, "e2": 15, 
    "e3": 15, "e4": 15,
    "e5": 7.5, "e6": 7.5,
    "e7": 15, "e8": 15, 
    "e9": 15, "e10": 15
}

# Function to generate random component states (0 or 1) based on failure probabilities
def generate_comps_st(probs):
    comps_st = {}

    # Define edge pairs that should have identical states
    edge_pairs = [("e1", "e2"), ("e3", "e4"), ("e5", "e6"), ("e7", "e8"), ("e9", "e10")]

    # Process edge pairs
    for e1, e2 in edge_pairs:
        state = np.random.choice([0, 1], p=[probs[e1][0], probs[e1][1]])  # Sample once
        comps_st[e1] = state
        comps_st[e2] = state  # Assign the same state to the paired edge

    return comps_st

comps_st = generate_comps_st(probs)
arc_capacity = {edge: intact_capacity[edge] * comps_st[edge] for edge in intact_capacity}

print("Generated Component States:")
for edge, state in comps_st.items():
    print(f"{edge}: {state}")

# Compute distances
def euclidean_distance(node1, node2):
    x1, y1 = nodes[node1]
    x2, y2 = nodes[node2]
    return round(((x2 - x1)**2 + (y2 - y1)**2)**0.5, 2)

arc_distance = {edge_name: euclidean_distance(u, v) for edge_name, (u, v) in edges.items()}   

# Create the graph
G = nx.DiGraph()

for node, position in nodes.items():
    G.add_node(node, pos=position)

for edge_name, (u, v) in edges.items(): 
    distance = arc_distance[edge_name]   
    capacity = arc_capacity[edge_name]  
    G.add_edge(u, v, weight=distance, capacity=capacity)   

# Calculate maximum allowable distance
# Demand data
demand = {
    'k1': {'origin': 'n1', 'destination': 'n4', 'amount': 28},
    'k2': {'origin': 'n2', 'destination': 'n3', 'amount': 18},
    'k3': {'origin': 'n1', 'destination': 'n3', 'amount': 11},
    'k4': {'origin': 'n3', 'destination': 'n4', 'amount': 16},
    'k5': {'origin': 'n4', 'destination': 'n1', 'amount': 20},
    'k6': {'origin': 'n3', 'destination': 'n2', 'amount': 14},
    'k7': {'origin': 'n3', 'destination': 'n1', 'amount': 11},
    'k8': {'origin': 'n4', 'destination': 'n3', 'amount': 18}
}

# Maximum allowable delay time 6 minutes / average velocity 149 km/h
avg_velo = 149
for commodity, info in demand.items():
    shortest_distance = nx.shortest_path_length(G, source=info['origin'], target=info['destination'], weight='weight')
    max_allowable_time = (shortest_distance * 60) / avg_velo + 6  # 6 minutes extra
    max_distance = max_allowable_time * avg_velo / 60

    print(f"\nCommodity: {commodity}")
    print(f"  Shortest distance: {shortest_distance} km")
    print(f"  Maximum allowable time: {max_allowable_time:.2f} minutes")
    print(f"  Maximum allowable distance: {max_distance:.2f} km")


# Plot the network
plt.figure(figsize=(8, 6))
pos = nx.get_node_attributes(G, 'pos')
nx.draw_networkx_nodes(G, pos, node_size=450, node_color="lightblue")
nx.draw_networkx_edges(
    G, pos,
    edgelist=list(edges.values()),  
    arrowstyle='-|>',
    arrowsize=15,
    connectionstyle='arc3,rad=0.1',
    min_target_margin=10,
    min_source_margin=10
)
edge_labels = {(u, v): f"{arc_distance[edge_name]} km, Cap: {arc_capacity[edge_name]}" 
               for edge_name, (u, v) in edges.items()}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, label_pos=0.5)
nx.draw_networkx_labels(G, pos, font_size=9)
plt.title("Network with Bidirectional Arcs")
plt.show()

# Print edge information
print("\nEdge Information:")
for edge_name, (u, v) in edges.items():
    edge_data = G.get_edge_data(u, v) 
    capacity = edge_data['capacity']
    print(f"{edge_name} Capacity: {capacity}")


Generated Component States:
e1: 1
e2: 1
e3: 1
e4: 1
e5: 0
e6: 0
e7: 1
e8: 1
e9: 1
e10: 1

Commodity: k1
  Shortest distance: 28.28 km
  Maximum allowable time: 17.39 minutes
  Maximum allowable distance: 43.18 km

Commodity: k2
  Shortest distance: 20.0 km
  Maximum allowable time: 14.05 minutes
  Maximum allowable distance: 34.90 km

Commodity: k3
  Shortest distance: 14.14 km
  Maximum allowable time: 11.69 minutes
  Maximum allowable distance: 29.04 km

Commodity: k4
  Shortest distance: 14.14 km
  Maximum allowable time: 11.69 minutes
  Maximum allowable distance: 29.04 km

Commodity: k5
  Shortest distance: 28.28 km
  Maximum allowable time: 17.39 minutes
  Maximum allowable distance: 43.18 km

Commodity: k6
  Shortest distance: 20.0 km
  Maximum allowable time: 14.05 minutes
  Maximum allowable distance: 34.90 km

Commodity: k7
  Shortest distance: 14.14 km
  Maximum allowable time: 11.69 minutes
  Maximum allowable distance: 29.04 km

Commodity: k8
  Shortest distance: 14.14 km


In [27]:
varis = {}
for k, v in edges.items():
    varis[k] = variable.Variable( name=k, values = [0, 1]) # values: edge flow capacity

In [28]:
def MCNF_systemfunc(arcs, comps_st, edges, arc_capacity, demand, max_distance, arc_distance):
    from gurobipy import Model, GRB, quicksum
    import networkx as nx

    # Create a graph for shortest path calculations
    G = nx.Graph()
    for e, (i, j) in edges.items():
        G.add_edge(i, j, weight=arc_distance.get(e, 1))  # Set edge weights based on distance

    # Create Gurobi optimization model
    model = Model("Network Flow Optimization")
    
    # 🚀 Gurobi 로그 숨기기
    model.setParam('OutputFlag', 0)  # ✅ 이 줄을 추가하면 Gurobi 메시지 안 나옴

    # Define variables
    flow = {}
    unmet_demand = {}

    for k, info in demand.items():
        unmet_demand[k] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"unsatisfied_{k}")
        for i, j in arcs:
            arc_key = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
            capacity = arc_capacity.get(arc_key, 0)
            flow[k, i, j] = model.addVar(lb=0, ub=capacity, vtype=GRB.CONTINUOUS, name=f"flow_{k}_{i}_{j}")  # 모든 경우에 대해 생성

    # Objective function: Minimize expected loss
    cost_coefficient = {
        k: info['amount'] * nx.shortest_path_length(G, source=info['origin'], target=info['destination'], weight='weight') * 0.1723
        for k, info in demand.items()
    }
    
    model.setObjective(
        quicksum(cost_coefficient[k] * unmet_demand[k] for k in demand),
        GRB.MINIMIZE
    )

    # Extract all nodes from edge values
    nodes = set(node for edge in edges.values() for node in edge)

    # Constraint 1: Flow conservation
    for k, info in demand.items():
        origin = info['origin']
        destination = info['destination']
        amount = info['amount']
        for node in nodes: 
            inflow = quicksum(flow[k, i, j] for i, j in arcs if j == node)
            outflow = quicksum(flow[k, i, j] for i, j in arcs if i == node)
            if node == origin:
                model.addConstr(outflow - inflow == amount - unmet_demand[k])
            elif node == destination:
                model.addConstr(outflow - inflow == - amount + unmet_demand[k])
            else:
                model.addConstr(outflow - inflow == 0)

    # Constraint 2: Arc capacity limits
    for i, j in arcs:
        arc_key = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
        model.addConstr(quicksum(flow[k, i, j] for k in demand if (k, i, j) in flow) <= arc_capacity.get(arc_key, 0))

    # Constraint 3: Distance limits
    for k, info in demand.items():
        origin = info['origin']
        distance_expr = quicksum(arc_distance.get(e, 0) * flow[k, i, j] for e, (i, j) in edges.items() if (k, i, j) in flow)
        total_flow = quicksum(flow[k, i, j] for i, j in arcs if (k, i, j) in flow and i == origin)
        model.addConstr(distance_expr <= max_distance * total_flow)

    # Perform optimization
    model.optimize()

    # Process results
    if model.status == GRB.OPTIMAL:
        expected_loss = model.objVal

        # Store flows in a structured dictionary
        flows = {
            k: {arc: flow[k, arc[0], arc[1]].X for arc in arcs} for k in demand
        }
        
        # Compute f_val (total satisfied demand per commodity)
        f_val = {
            k: sum(flow[k, i, j].X for i, j in arcs if i == demand[k]['origin']) for k in demand
        }

        # Construct f_dict (actual flows for each edge)
        f_dict = {
            (i, j): sum(flow[k, i, j].X for k in demand) for i, j in arcs
        }

        if expected_loss <= 4400:
            sys_st = 's'

            # Minimum survival rule extraction
            min_comps_st = {}  # ✅ 기존 함수(`net_conn`)과 동일한 데이터 구조 유지
            for k, x in comps_st.items():
                if k in edges:
                    i, j = edges[k]
                    k_flow = max(f_dict.get((i, j), 0), f_dict.get((j, i), 0))  # Check both directions
                    if k_flow > 0:  
                        min_comps_st[k] = 1  # ✅ `{k: 1, ...}` 형태 유지

            return expected_loss, sys_st, min_comps_st  # ✅ 기존 시스템 함수 형식과 동일

        else:
            sys_st = 'f'
            return expected_loss, sys_st, None  # ✅ 실패한 경우 `min_comps_st = None` 유지

    else:
        return None, None, None  # ✅ 최적해가 없는 경우 `None` 반환

In [29]:
sys_fun = lambda comps_st: MCNF_systemfunc(
    arcs=arcs,
    comps_st=comps_st,
    edges=edges,
    arc_capacity={edge: intact_capacity[edge] * comps_st[edge] for edge in intact_capacity},  # ✅ 동적으로 계산
    demand=demand,
    max_distance=max_distance,
    arc_distance=arc_distance
)


# Print input values
print("\n🔹 Input Values:")
print("Component States (comps_st):", comps_st)
print("Edges:", edges)
print("Arc Capacity:", arc_capacity)
print("Demand:", demand)
print("Max Distance:", max_distance)
print("Arc Distance:", arc_distance)

# Run the function and capture outputs
sys_st, min_comps_st, f_val = MCNF_systemfunc(arcs, comps_st, edges, arc_capacity, demand, max_distance, arc_distance)

# Print output values
print("\n🔹 Output Values:")
print("System State:", sys_st)
print("Minimum component state:", min_comps_st)
print("Flow values per commodity:", f_val)  # 각 commodity별 총 유량


🔹 Input Values:
Component States (comps_st): {'e1': np.int64(1), 'e2': np.int64(1), 'e3': np.int64(1), 'e4': np.int64(1), 'e5': np.int64(0), 'e6': np.int64(0), 'e7': np.int64(1), 'e8': np.int64(1), 'e9': np.int64(1), 'e10': np.int64(1)}
Edges: {'e1': ('n1', 'n2'), 'e2': ('n2', 'n1'), 'e3': ('n1', 'n3'), 'e4': ('n3', 'n1'), 'e5': ('n2', 'n3'), 'e6': ('n3', 'n2'), 'e7': ('n2', 'n4'), 'e8': ('n4', 'n2'), 'e9': ('n3', 'n4'), 'e10': ('n4', 'n3')}
Arc Capacity: {'e1': np.int64(15), 'e2': np.int64(15), 'e3': np.int64(15), 'e4': np.int64(15), 'e5': np.float64(0.0), 'e6': np.float64(0.0), 'e7': np.int64(15), 'e8': np.int64(15), 'e9': np.int64(15), 'e10': np.int64(15)}
Demand: {'k1': {'origin': 'n1', 'destination': 'n4', 'amount': 28}, 'k2': {'origin': 'n2', 'destination': 'n3', 'amount': 18}, 'k3': {'origin': 'n1', 'destination': 'n3', 'amount': 11}, 'k4': {'origin': 'n3', 'destination': 'n4', 'amount': 16}, 'k5': {'origin': 'n4', 'destination': 'n1', 'amount': 20}, 'k6': {'origin': 'n3', 'des

In [30]:
# Run BRC optimization
brs, rules, sys_res, monitor = brc.run(varis, probs, sys_fun, max_sf=10000, max_nb=10000, pf_bnd_wr=0.0)

# Print branch details with expected loss calculation
print("\n🔹 All Branches and Expected Loss:")

for i, branch in enumerate(brs, 1):
    expected_loss_down, _, _ = sys_fun(branch.down) 
    expected_loss_up, _, _ = sys_fun(branch.up)

    print(f"\nBranch {i}:")
    print(f"  Down State: {branch.down}")
    print(f"  Up State: {branch.up}")
    print(f"  Probability: {branch.p}")
    print(f"  Expected Loss (Down State): {expected_loss_down:.4f}")
    print(f"  Expected Loss (Up State): {expected_loss_up:.4f}")

[System function runs 200]..
The # of found non-dominated rules (f, s): 2 (1, 1)
Probability of branchs (f, s, u): (1.0000e-01, 6.34e-01, 2.6625e-01)
The # of branches (f, s, u), (min, avg) len of rf: 11 (1, 1, 9), (1, 1.00)
[System function runs 400]..
The # of found non-dominated rules (f, s): 2 (1, 1)
Probability of branchs (f, s, u): (1.0000e-01, 6.34e-01, 2.6625e-01)
The # of branches (f, s, u), (min, avg) len of rf: 11 (1, 1, 9), (1, 1.00)
[System function runs 600]..
The # of found non-dominated rules (f, s): 2 (1, 1)
Probability of branchs (f, s, u): (1.0000e-01, 6.34e-01, 2.6625e-01)
The # of branches (f, s, u), (min, avg) len of rf: 11 (1, 1, 9), (1, 1.00)
[System function runs 800]..
The # of found non-dominated rules (f, s): 2 (1, 1)
Probability of branchs (f, s, u): (1.0000e-01, 6.34e-01, 2.6625e-01)
The # of branches (f, s, u), (min, avg) len of rf: 11 (1, 1, 9), (1, 1.00)
[System function runs 1000]..
The # of found non-dominated rules (f, s): 2 (1, 1)
Probability of bra